In [1]:
import boto3

# Get the service resource
sqs = boto3.resource('sqs')

# Create the queue. This returns an SQS.Queue instance
queue = sqs.create_queue(QueueName='biomatch_tst', Attributes={'DelaySeconds': '5'})

# You can now access identifiers and attributes
print(queue.url)
print(queue.attributes.get('DelaySeconds'))

https://queue.amazonaws.com/127516845550/biomatch_tst
5


In [3]:

queue2 = sqs.get_queue_by_name(QueueName='biomatch_tst')

# You can now access identifiers and attributes
print(queue2.url)
print(queue2.attributes.get('DelaySeconds'))

https://queue.amazonaws.com/127516845550/biomatch_tst
5


In [12]:
sqs_west = boto3.resource('sqs', region_name='us-west-2')
queue3 = sqs_west.get_queue_by_name(QueueName='ir_queue_dev')
print(queue3.url)
#print(queue3.attributes.get('DelaySeconds'))
print len(queue.receive_messages())

https://us-west-2.queue.amazonaws.com/127516845550/ir_queue_dev
0


In [4]:
# Print out each queue name, which is part of its ARN
for queue in sqs.queues.all():
    print(queue.url)

https://queue.amazonaws.com/127516845550/biomatch_tst
https://queue.amazonaws.com/127516845550/ed_ta_dev
https://queue.amazonaws.com/127516845550/fut_treatment_arm_queue
https://queue.amazonaws.com/127516845550/lw_patient_development
https://queue.amazonaws.com/127516845550/lw_treatment_arm_development
https://queue.amazonaws.com/127516845550/lw_treatment_arm_test
https://queue.amazonaws.com/127516845550/patient_development
https://queue.amazonaws.com/127516845550/patient_events_test_raseel
https://queue.amazonaws.com/127516845550/patient_processor_dev_ramaniv
https://queue.amazonaws.com/127516845550/patient_processor_dev_zakharovrv
https://queue.amazonaws.com/127516845550/patient_processor_test_raseel
https://queue.amazonaws.com/127516845550/rm_patient_development
https://queue.amazonaws.com/127516845550/rm_treatment_arm_development
https://queue.amazonaws.com/127516845550/rm_treatment_arm_test
https://queue.amazonaws.com/127516845550/se_patient_dev
https://queue.amazonaws.com/1275168

In [5]:
response = queue2.send_message(MessageBody='world')

# The response is NOT a resource, but gives you a message ID and MD5
print(response.get('MessageId'))
print(response.get('MD5OfMessageBody'))
print response

c7ff87c0-25c1-4821-b0e2-61884b687488
7d793037a0760186574b0282f2f435e7
{u'MD5OfMessageBody': '7d793037a0760186574b0282f2f435e7', 'ResponseMetadata': {'HTTPStatusCode': 200, 'RequestId': '5a73f62f-d0a1-5bc9-8fd1-a60aa9ca0bf6'}, u'MessageId': 'c7ff87c0-25c1-4821-b0e2-61884b687488'}


In [6]:
response = queue2.send_message(MessageBody='boto3', MessageAttributes={
    'Author': {
        'StringValue': 'Daniel',
        'DataType': 'String'
    }
})
print response

{u'MD5OfMessageBody': '6686853da3491a56c98917cc5c4ddea2', u'MD5OfMessageAttributes': 'd57be2825f78cdc7112c0f9dc4b21231', 'ResponseMetadata': {'HTTPStatusCode': 200, 'RequestId': '33541df6-d073-5ecf-b558-4dd3aa62238f'}, u'MessageId': '9f792718-43b0-4b3b-a744-5c1f3e0dd19b'}


In [7]:
print 'hello world'

hello world


In [15]:
patient_id = request.args.get['patient_id']

message_body = '''{
  "patient_id": %s,
  "molecular_id": %s,
  "analysis_id": "job2",
  "s3_bucket_name": "pedmatch-demo/3344/3344-bsn-msn-blood/job2",
  "tsv_file_path_name": "3344-blood.tsv",
  "vcf_file_path_name": "3344-blood.vcf",
  "dna_bam_file_path_name": "dna.bam",
  "cdna_bam_file_path_name": "cdna.bam",
  "dna_bai_file_path_name": "dna.bam.bai",
  "cdna_bai_file_path_name": "cdna.bam.bai"
}'''%{patient_id, molecular_id}
response = queue3.send_message(MessageBody=message_body)
print response

{u'MD5OfMessageBody': 'd8412be06dc94465601bd0c7a48909f8', 'ResponseMetadata': {'HTTPStatusCode': 200, 'RequestId': 'cdf4601a-134f-5bc1-9883-303f6d91fbdf'}, u'MessageId': '1d58aa92-bbee-4f32-9e7c-a7d5ab0cd63f'}


In [18]:
for message in queue3.receive_messages():
    print message.body
    print message.message_attributes

{
  "patient_id": "3344",
  "molecular_id": "3344-bsn-msn-blood",
  "analysis_id": "job2",
  "s3_bucket_name": "pedmatch-demo/3344/3344-bsn-msn-blood/job2",
  "tsv_file_path_name": "3344-blood.tsv",
  "vcf_file_path_name": "3344-blood.vcf",
  "dna_bam_file_path_name": "dna.bam",
  "cdna_bam_file_path_name": "cdna.bam",
  "dna_bai_file_path_name": "dna.bam.bai",
  "cdna_bai_file_path_name": "cdna.bam.bai"
}
None


In [20]:
response = queue3.send_messages(Entries=[
    {
        'Id': '1',
        'MessageBody': 'qqss1'
    },
    {
        'Id': '2',
        'MessageBody': 'qqss2',
        'MessageAttributes': {
            'Author': {
                'StringValue': 'Daniel',
                'DataType': 'String'
            }
        }
    }
])

# Print out any failures
print(response.get('Failed'))

None


In [21]:
# Process messages by printing out body and optional author name, then delete it
for message in queue3.receive_messages(MessageAttributeNames=['Author']):
    # Get the custom author message attribute if it was set
    author_text = ''
    if message.message_attributes is not None:
        author_name = message.message_attributes.get('Author').get('StringValue')
        if author_name:
            author_text = ' ({0})'.format(author_name)

    # Print out the body and author (if set)
    print('Hello, {0}!{1}'.format(message.body, author_text))

    # Let the queue know that the message is processed
    message.delete()

Hello, {
  "patient_id": "3344",
  "molecular_id": "3344-bsn-msn-blood",
  "analysis_id": "job2",
  "s3_bucket_name": "pedmatch-demo/3344/3344-bsn-msn-blood/job2",
  "tsv_file_path_name": "3344-blood.tsv",
  "vcf_file_path_name": "3344-blood.vcf",
  "dna_bam_file_path_name": "dna.bam",
  "cdna_bam_file_path_name": "cdna.bam",
  "dna_bai_file_path_name": "dna.bam.bai",
  "cdna_bai_file_path_name": "cdna.bam.bai"
}!


In [22]:
print len(queue3.receive_messages())

1


In [23]:
for message in queue3.receive_messages():
    print message.body
    message.delete()


{
  "patient_id": "3344",
  "molecular_id": "3344-bsn-msn-blood",
  "analysis_id": "job2",
  "s3_bucket_name": "pedmatch-demo/3344/3344-bsn-msn-blood/job2",
  "tsv_file_path_name": "3344-blood.tsv",
  "vcf_file_path_name": "3344-blood.vcf",
  "dna_bam_file_path_name": "dna.bam",
  "cdna_bam_file_path_name": "cdna.bam",
  "dna_bai_file_path_name": "dna.bam.bai",
  "cdna_bai_file_path_name": "cdna.bam.bai"
}


In [24]:
print len(queue3.receive_messages())

1
